### Chatbot

In [1]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

Using TensorFlow backend.
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
df = pd.read_excel(r"training_chatbot.xlsx")

Intent type              Sentence
0    Greeting                   Hi!
1    Greeting                 Hello
2    Greeting          How are you?
3    Greeting  Hi, how is it going?
4    Greeting            Greetings!

In [3]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [6]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [25]:
def processing(df, pretreatment = False, Tfidf = True, cv = None, stopwords = []):
  # Normalizamos y limpiamos el corpus 
  if pretreatment == True:
    df["Sentence"] = df['Sentence'].apply(lambda x: word_treatment(x))
    print("El corpus ha sido pretratado")
    
     # Transformamos nuestro corpus a un vector Tfidf
  if Tfidf == True:

    if cv == None:
      cv = TfidfVectorizer(
        stop_words= stopwords,
        ngram_range=(1, 4),
        strip_accents='ascii',
        max_df=0.99,
        min_df=0,
        max_features=100
      )
      cv.fit(df["Sentence"])
      X = cv.transform(df["Sentence"])
      print("Se ha realizado una vectorización Tfidf")
      return df, X, cv

    else:
      X = cv.transform(df["Sentence"])
      #print("Se ha realizado una vectorización Tfidf basado en el corpus suministrado por cv")
    return df, X
  else:
    return df


In [9]:
df_train, X_train, cv = processing(df_train)
df_test, X_test = processing(df_test, cv = cv)

Se ha realizado una vectorización Tfidf
Se ha realizado una vectorización Tfidf basado en el corpus suministrado por cv


In [10]:
y_train = df_train[["Greeting","Search","Suggestions"]]
y_test = df_test[["Greeting","Search","Suggestions"]]

In [11]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.7, 
test_size=0.3, random_state=42, shuffle=True, stratify=df_train["Intent type"])

In [12]:
shape= X_train.shape[1]

X_train.sort_indices()
X_validation.sort_indices()

In [14]:
def mlp_greeting(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [15]:
mlp_greeting = mlp_greeting(shape)
history = mlp_greeting.fit(X_train, np.asarray(y_train["Greeting"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Greeting"]).reshape(-1,1)),
    epochs=200,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 53 samples, validate on 23 samples
Epoch 1/200
 - 4s - loss: 0.6572 - acc: 0.6604 - val_loss: 0.6767 - val_acc: 0.7391
Epoch 2/200
 - 0s - loss: 0.6515 - acc: 0.6415 - val_loss: 0.6746 - val_acc: 0.7391
Epoch 3/200
 - 0s - loss: 0.6707 - acc: 0.5660 - val_loss: 0.6724 - val_acc: 0.7391
Epoch 4/200
 - 0s - loss: 0.6658 - acc: 0.6604 - val_loss: 0.6699 - val_acc: 0.7391
Epoch 5/200
 - 0s - loss: 0.6344 - acc: 0.6981 - val_loss: 0.6673 - val_acc: 0.7391
Epoch 6/200
 - 0s - loss: 0.6466 - acc: 0.5849 - val_loss: 0.6648 - val_acc: 0.7391
Epoch 7/200
 - 0s - loss: 0.6649 - acc: 0.6415 - val_loss: 0.6622 - val_acc: 0.7391
Epoch 8/200
 - 0s - loss: 0.6417 - acc: 0.7170 - val_loss: 0.6596 - val_acc: 0.7391
Epoch 9/200
 - 0s - loss: 0.6254 - acc: 0.6792 - val_loss: 0.6570 - val_acc: 0.7391
Epoch 10/200
 - 0s - loss: 0.6258 - acc: 0.6981 - val_loss: 0.6544 - val_acc: 0.7391
Epoch 11/200
 - 0s - loss: 0.6323 - acc: 0.6604 - val_loss: 0.6518 - val_acc: 0.7391
Epoch 12/200
 - 0s - loss: 0.6

Epoch 97/200
 - 0s - loss: 0.4387 - acc: 0.7547 - val_loss: 0.4868 - val_acc: 0.7391
Epoch 98/200
 - 0s - loss: 0.4723 - acc: 0.7736 - val_loss: 0.4852 - val_acc: 0.7391
Epoch 99/200
 - 0s - loss: 0.4875 - acc: 0.7547 - val_loss: 0.4836 - val_acc: 0.7391
Epoch 100/200
 - 0s - loss: 0.3975 - acc: 0.8113 - val_loss: 0.4822 - val_acc: 0.7391
Epoch 101/200
 - 0s - loss: 0.4233 - acc: 0.7736 - val_loss: 0.4807 - val_acc: 0.7391
Epoch 102/200
 - 0s - loss: 0.4264 - acc: 0.7736 - val_loss: 0.4791 - val_acc: 0.7391
Epoch 103/200
 - 0s - loss: 0.4355 - acc: 0.7925 - val_loss: 0.4775 - val_acc: 0.7391
Epoch 104/200
 - 0s - loss: 0.4245 - acc: 0.7736 - val_loss: 0.4758 - val_acc: 0.7391
Epoch 105/200
 - 0s - loss: 0.4133 - acc: 0.7547 - val_loss: 0.4744 - val_acc: 0.7391
Epoch 106/200
 - 0s - loss: 0.4320 - acc: 0.7358 - val_loss: 0.4730 - val_acc: 0.7391
Epoch 107/200
 - 0s - loss: 0.4757 - acc: 0.7358 - val_loss: 0.4715 - val_acc: 0.7391
Epoch 108/200
 - 0s - loss: 0.4148 - acc: 0.7170 - val_lo

Epoch 193/200
 - 0s - loss: 0.2538 - acc: 0.8302 - val_loss: 0.4143 - val_acc: 0.8261
Epoch 194/200
 - 0s - loss: 0.2720 - acc: 0.8113 - val_loss: 0.4151 - val_acc: 0.8261
Epoch 195/200
 - 0s - loss: 0.2568 - acc: 0.8302 - val_loss: 0.4156 - val_acc: 0.8261
Epoch 196/200
 - 0s - loss: 0.2326 - acc: 0.8113 - val_loss: 0.4161 - val_acc: 0.8261
Epoch 197/200
 - 0s - loss: 0.2595 - acc: 0.8113 - val_loss: 0.4169 - val_acc: 0.8261
Epoch 198/200
 - 0s - loss: 0.2101 - acc: 0.9245 - val_loss: 0.4176 - val_acc: 0.8261
Epoch 199/200
 - 0s - loss: 0.2290 - acc: 0.8491 - val_loss: 0.4184 - val_acc: 0.8261
Epoch 200/200
 - 0s - loss: 0.2498 - acc: 0.8302 - val_loss: 0.4193 - val_acc: 0.8261


In [16]:
def mlp_search(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [17]:
mlp_search = mlp_search(shape)
history = mlp_search.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 53 samples, validate on 23 samples
Epoch 1/250
 - 1s - loss: 0.6908 - acc: 0.5660 - val_loss: 0.6915 - val_acc: 0.7391
Epoch 2/250
 - 0s - loss: 0.6832 - acc: 0.7358 - val_loss: 0.6888 - val_acc: 0.7391
Epoch 3/250
 - 0s - loss: 0.6851 - acc: 0.7358 - val_loss: 0.6859 - val_acc: 0.7391
Epoch 4/250
 - 0s - loss: 0.6831 - acc: 0.7358 - val_loss: 0.6832 - val_acc: 0.7391
Epoch 5/250
 - 0s - loss: 0.6923 - acc: 0.7358 - val_loss: 0.6807 - val_acc: 0.7391
Epoch 6/250
 - 0s - loss: 0.6929 - acc: 0.7358 - val_loss: 0.6780 - val_acc: 0.7391
Epoch 7/250
 - 0s - loss: 0.6764 - acc: 0.7358 - val_loss: 0.6756 - val_acc: 0.7391
Epoch 8/250
 - 0s - loss: 0.6815 - acc: 0.7358 - val_loss: 0.6732 - val_acc: 0.7391
Epoch 9/250
 - 0s - loss: 0.6908 - acc: 0.7358 - val_loss: 0.6709 - val_acc: 0.7391
Epoch 10/250
 - 0s - loss: 0.6774 - acc: 0.7358 - val_loss: 0.6685 - val_acc: 0.7391
Epoch 11/250
 - 0s - loss: 0.6800 - acc: 0.7358 - val_loss: 0.6662 - val_acc: 0.7391
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.4327 - acc: 0.7358 - val_loss: 0.4621 - val_acc: 0.7391
Epoch 98/250
 - 0s - loss: 0.3580 - acc: 0.7358 - val_loss: 0.4629 - val_acc: 0.7391
Epoch 99/250
 - 0s - loss: 0.3850 - acc: 0.7358 - val_loss: 0.4637 - val_acc: 0.7391
Epoch 100/250
 - 0s - loss: 0.3737 - acc: 0.7358 - val_loss: 0.4646 - val_acc: 0.7391
Epoch 101/250
 - 0s - loss: 0.3552 - acc: 0.7358 - val_loss: 0.4655 - val_acc: 0.7391
Epoch 102/250
 - 0s - loss: 0.3789 - acc: 0.7358 - val_loss: 0.4663 - val_acc: 0.7391
Epoch 103/250
 - 0s - loss: 0.3504 - acc: 0.7358 - val_loss: 0.4674 - val_acc: 0.7391
Epoch 104/250
 - 0s - loss: 0.3691 - acc: 0.7358 - val_loss: 0.4684 - val_acc: 0.7391
Epoch 105/250
 - 0s - loss: 0.3700 - acc: 0.7358 - val_loss: 0.4694 - val_acc: 0.7391
Epoch 106/250
 - 0s - loss: 0.3503 - acc: 0.7358 - val_loss: 0.4707 - val_acc: 0.7391
Epoch 107/250
 - 0s - loss: 0.3690 - acc: 0.7358 - val_loss: 0.4719 - val_acc: 0.7391
Epoch 108/250
 - 0s - loss: 0.3437 - acc: 0.7358 - val_lo

Epoch 193/250
 - 0s - loss: 0.2457 - acc: 0.9245 - val_loss: 0.6835 - val_acc: 0.8261
Epoch 194/250
 - 0s - loss: 0.2323 - acc: 0.9245 - val_loss: 0.6873 - val_acc: 0.8261
Epoch 195/250
 - 0s - loss: 0.2582 - acc: 0.8868 - val_loss: 0.6910 - val_acc: 0.8261
Epoch 196/250
 - 0s - loss: 0.2434 - acc: 0.9434 - val_loss: 0.6947 - val_acc: 0.8261
Epoch 197/250
 - 0s - loss: 0.2535 - acc: 0.9434 - val_loss: 0.6985 - val_acc: 0.8261
Epoch 198/250
 - 0s - loss: 0.2475 - acc: 0.8868 - val_loss: 0.7019 - val_acc: 0.8261
Epoch 199/250
 - 0s - loss: 0.2530 - acc: 0.8679 - val_loss: 0.7050 - val_acc: 0.8261
Epoch 200/250
 - 0s - loss: 0.2510 - acc: 0.9057 - val_loss: 0.7074 - val_acc: 0.8261
Epoch 201/250
 - 0s - loss: 0.2185 - acc: 0.8868 - val_loss: 0.7092 - val_acc: 0.8261
Epoch 202/250
 - 0s - loss: 0.2565 - acc: 0.8868 - val_loss: 0.7106 - val_acc: 0.8261
Epoch 203/250
 - 0s - loss: 0.2422 - acc: 0.8868 - val_loss: 0.7126 - val_acc: 0.8261
Epoch 204/250
 - 0s - loss: 0.2517 - acc: 0.9245 - val

In [18]:
def mlp_suggestion(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer = opt)
# return our model
    return model

In [19]:
mlp_suggestion = mlp_suggestion(shape)
history = mlp_suggestion.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 53 samples, validate on 23 samples
Epoch 1/250
 - 1s - loss: 0.6945 - acc: 0.5283 - val_loss: 0.6956 - val_acc: 0.6522
Epoch 2/250
 - 0s - loss: 0.6925 - acc: 0.6415 - val_loss: 0.6949 - val_acc: 0.6087
Epoch 3/250
 - 0s - loss: 0.6898 - acc: 0.6415 - val_loss: 0.6941 - val_acc: 0.6087
Epoch 4/250
 - 0s - loss: 0.6917 - acc: 0.6604 - val_loss: 0.6933 - val_acc: 0.6957
Epoch 5/250
 - 0s - loss: 0.6883 - acc: 0.5472 - val_loss: 0.6924 - val_acc: 0.6957
Epoch 6/250
 - 0s - loss: 0.6942 - acc: 0.5660 - val_loss: 0.6915 - val_acc: 0.6957
Epoch 7/250
 - 0s - loss: 0.6853 - acc: 0.6792 - val_loss: 0.6903 - val_acc: 0.6957
Epoch 8/250
 - 0s - loss: 0.6849 - acc: 0.6981 - val_loss: 0.6890 - val_acc: 0.6957
Epoch 9/250
 - 0s - loss: 0.6860 - acc: 0.7170 - val_loss: 0.6877 - val_acc: 0.6957
Epoch 10/250
 - 0s - loss: 0.6875 - acc: 0.6792 - val_loss: 0.6868 - val_acc: 0.6957
Epoch 11/250
 - 0s - loss: 0.6839 - acc: 0.6792 - val_loss: 0.6854 - val_acc: 0.6957
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.4512 - acc: 0.7358 - val_loss: 0.4611 - val_acc: 0.7391
Epoch 98/250
 - 0s - loss: 0.4532 - acc: 0.7358 - val_loss: 0.4597 - val_acc: 0.7391
Epoch 99/250
 - 0s - loss: 0.4357 - acc: 0.7358 - val_loss: 0.4585 - val_acc: 0.7391
Epoch 100/250
 - 0s - loss: 0.4508 - acc: 0.7358 - val_loss: 0.4575 - val_acc: 0.7391
Epoch 101/250
 - 0s - loss: 0.4423 - acc: 0.7358 - val_loss: 0.4565 - val_acc: 0.7391
Epoch 102/250
 - 0s - loss: 0.4514 - acc: 0.7358 - val_loss: 0.4555 - val_acc: 0.7391
Epoch 103/250
 - 0s - loss: 0.4103 - acc: 0.7358 - val_loss: 0.4545 - val_acc: 0.7391
Epoch 104/250
 - 0s - loss: 0.4555 - acc: 0.7358 - val_loss: 0.4538 - val_acc: 0.7391
Epoch 105/250
 - 0s - loss: 0.4222 - acc: 0.7358 - val_loss: 0.4531 - val_acc: 0.7391
Epoch 106/250
 - 0s - loss: 0.3956 - acc: 0.7358 - val_loss: 0.4523 - val_acc: 0.7391
Epoch 107/250
 - 0s - loss: 0.4150 - acc: 0.7358 - val_loss: 0.4513 - val_acc: 0.7391
Epoch 108/250
 - 0s - loss: 0.4229 - acc: 0.7358 - val_lo

Epoch 193/250
 - 0s - loss: 0.3331 - acc: 0.7358 - val_loss: 0.4605 - val_acc: 0.7391
Epoch 194/250
 - 0s - loss: 0.3616 - acc: 0.7358 - val_loss: 0.4620 - val_acc: 0.7391
Epoch 195/250
 - 0s - loss: 0.3550 - acc: 0.7358 - val_loss: 0.4635 - val_acc: 0.7391
Epoch 196/250
 - 0s - loss: 0.3491 - acc: 0.7358 - val_loss: 0.4648 - val_acc: 0.7391
Epoch 197/250
 - 0s - loss: 0.3121 - acc: 0.7358 - val_loss: 0.4660 - val_acc: 0.7391
Epoch 198/250
 - 0s - loss: 0.3316 - acc: 0.7358 - val_loss: 0.4666 - val_acc: 0.7391
Epoch 199/250
 - 0s - loss: 0.3721 - acc: 0.7358 - val_loss: 0.4670 - val_acc: 0.7391
Epoch 200/250
 - 0s - loss: 0.2921 - acc: 0.7358 - val_loss: 0.4675 - val_acc: 0.7391
Epoch 201/250
 - 0s - loss: 0.3276 - acc: 0.7358 - val_loss: 0.4678 - val_acc: 0.7391
Epoch 202/250
 - 0s - loss: 0.3307 - acc: 0.7358 - val_loss: 0.4680 - val_acc: 0.7391
Epoch 203/250
 - 0s - loss: 0.3824 - acc: 0.7358 - val_loss: 0.4682 - val_acc: 0.7391
Epoch 204/250
 - 0s - loss: 0.2963 - acc: 0.7358 - val

In [20]:
def mlp_farewell(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [21]:
mlp_farewell = mlp_farewell(shape)
history = mlp_farewell.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 53 samples, validate on 23 samples
Epoch 1/250
 - 1s - loss: 0.6915 - acc: 0.7547 - val_loss: 0.6928 - val_acc: 0.7391
Epoch 2/250
 - 0s - loss: 0.6929 - acc: 0.7170 - val_loss: 0.6923 - val_acc: 0.7391
Epoch 3/250
 - 0s - loss: 0.6928 - acc: 0.7170 - val_loss: 0.6919 - val_acc: 0.7391
Epoch 4/250
 - 0s - loss: 0.6943 - acc: 0.6415 - val_loss: 0.6914 - val_acc: 0.7391
Epoch 5/250
 - 0s - loss: 0.6922 - acc: 0.6981 - val_loss: 0.6909 - val_acc: 0.7391
Epoch 6/250
 - 0s - loss: 0.6904 - acc: 0.7358 - val_loss: 0.6904 - val_acc: 0.7391
Epoch 7/250
 - 0s - loss: 0.6909 - acc: 0.7358 - val_loss: 0.6899 - val_acc: 0.7391
Epoch 8/250
 - 0s - loss: 0.6900 - acc: 0.6981 - val_loss: 0.6894 - val_acc: 0.7391
Epoch 9/250
 - 0s - loss: 0.6884 - acc: 0.6981 - val_loss: 0.6889 - val_acc: 0.7391
Epoch 10/250
 - 0s - loss: 0.6863 - acc: 0.7925 - val_loss: 0.6884 - val_acc: 0.7391
Epoch 11/250
 - 0s - loss: 0.6887 - acc: 0.7170 - val_loss: 0.6879 - val_acc: 0.7391
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.5954 - acc: 0.8491 - val_loss: 0.6532 - val_acc: 0.6522
Epoch 98/250
 - 0s - loss: 0.6044 - acc: 0.8113 - val_loss: 0.6529 - val_acc: 0.6522
Epoch 99/250
 - 0s - loss: 0.6038 - acc: 0.8679 - val_loss: 0.6526 - val_acc: 0.6522
Epoch 100/250
 - 0s - loss: 0.5745 - acc: 0.8868 - val_loss: 0.6523 - val_acc: 0.6522
Epoch 101/250
 - 0s - loss: 0.5877 - acc: 0.8302 - val_loss: 0.6519 - val_acc: 0.6522
Epoch 102/250
 - 0s - loss: 0.5968 - acc: 0.8113 - val_loss: 0.6515 - val_acc: 0.6522
Epoch 103/250
 - 0s - loss: 0.5774 - acc: 0.9057 - val_loss: 0.6511 - val_acc: 0.6522
Epoch 104/250
 - 0s - loss: 0.5768 - acc: 0.9057 - val_loss: 0.6509 - val_acc: 0.6522
Epoch 105/250
 - 0s - loss: 0.6047 - acc: 0.8302 - val_loss: 0.6506 - val_acc: 0.6522
Epoch 106/250
 - 0s - loss: 0.5676 - acc: 0.8679 - val_loss: 0.6501 - val_acc: 0.6957
Epoch 107/250
 - 0s - loss: 0.5600 - acc: 0.8868 - val_loss: 0.6496 - val_acc: 0.6957
Epoch 108/250
 - 0s - loss: 0.5855 - acc: 0.8302 - val_lo

Epoch 193/250
 - 0s - loss: 0.3789 - acc: 0.9434 - val_loss: 0.6551 - val_acc: 0.7826
Epoch 194/250
 - 0s - loss: 0.3399 - acc: 0.9811 - val_loss: 0.6565 - val_acc: 0.7826
Epoch 195/250
 - 0s - loss: 0.3935 - acc: 0.9245 - val_loss: 0.6580 - val_acc: 0.7826
Epoch 196/250
 - 0s - loss: 0.3726 - acc: 0.9434 - val_loss: 0.6594 - val_acc: 0.7826
Epoch 197/250
 - 0s - loss: 0.3560 - acc: 0.9434 - val_loss: 0.6608 - val_acc: 0.7826
Epoch 198/250
 - 0s - loss: 0.3302 - acc: 0.9811 - val_loss: 0.6622 - val_acc: 0.7826
Epoch 199/250
 - 0s - loss: 0.3705 - acc: 0.9811 - val_loss: 0.6640 - val_acc: 0.7826
Epoch 200/250
 - 0s - loss: 0.3662 - acc: 0.9434 - val_loss: 0.6660 - val_acc: 0.7826
Epoch 201/250
 - 0s - loss: 0.3583 - acc: 0.9623 - val_loss: 0.6678 - val_acc: 0.7826
Epoch 202/250
 - 0s - loss: 0.3385 - acc: 0.9811 - val_loss: 0.6698 - val_acc: 0.7826
Epoch 203/250
 - 0s - loss: 0.3898 - acc: 0.9434 - val_loss: 0.6715 - val_acc: 0.7826
Epoch 204/250
 - 0s - loss: 0.3396 - acc: 0.9811 - val

# CHATBOT TEST

In [23]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)

    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob]
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    else:
        print("Esto es una sugerencia")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [35]:
chatbot()

Tell me things related to Napoleon
Esto es un saludo


* Correctamente
 * La mayoría de saludos
 * I want to know about Data Science (búsqueda)
 * I want to know things connected to la Sagrada Familia (sugerencia)
 * Watcha doing'? (saludo)
 * Tell me things related to Data Science (sugerencia)

* Incorrectamente
 * What is Data Science (sugerencia)
 * Tell me things related to Napoleon (saludo)
 * Who is Cristobal Colon (saludo)



* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header